In [1]:
import json, csv, re, time
from collections import defaultdict
from collections import OrderedDict
from organ_utils import loadOrganData
from pymysql.converters import escape_string

## Load organ data

In [2]:
# load organ data and add to global variables
organdata = loadOrganData()
for key in organdata.keys():
    globals()[key] = organdata[key]

print(organdata.keys())

dict_keys(['organids', 'base', 'history_base', 'history_projects', 'dispositions', 'compoundstops', 'tech', 'texts_hist', 'texts_fulltexts', 'texts_tech', 'texts_kunst', 'texts_bijzonderheden', 'texts_offsets'])


In [3]:
#Dates of the volumes of the organencyclopaedia. These will be taken as the dates for the 'current' dispositions
part2date = {
    "Part01": 1997,
    "Part02": 1997,
    "Part03": 1999,
    "Part04": 1999,
    "Part05": 2001,
    "Part06": 2002,
    "Part07": 2002,
    "Part08": 2003,
    "Part09": 2004,
    "Part10": 2005,
    "Part11": 2006,
    "Part12": 2007,
    "Part13": 2008,
    "Part14": 2010,
    "Part15": 2010
}

## Some helper functions

In [4]:
def text2html(text):
    return "<p>"+text.replace('&','&amp;').replace('\n\n','</p>\n<p>&nbsp;</p>\n<p>').replace('\n','<br />')

def toSQLstring(text):
    return "'" + escape_string(text) + "'"

# "tussen xxxx en yyyy"
# "xxxx of yyyy"
# "xxxx/yyyy"
# "xxxx/yyyy/zzzz"
# "ca xxxx" ("ca." does not occur)
# "xxxx-yyyy"
# "xxxx en yyyy"
# "xxxx, yyyy en zzzz" (never a "," after "yyyy". [0-9]{4}, [0-9]{4} en [0-9]{4} occurs 2x )
def getSortingYear_project(date: str) -> str:
    year_re = re.compile(r'[12][0-9]{3}')
    year_match = re.findall(year_re, date)
    if not year_match:
        return '0000'
    #TODO:
    # 'of', '/', 'tussen', 'ca', '-'
    return year_match[0]

#for now: just take the last year mentioned.
#In case of 'voor xxxx' or vóór xxxx', take xxxx - 1
#If no year is given, take 0
def getSortingYear_disposition(organid, date: str) -> str:
    year_re = re.compile(r'[12][0-9]{3}')
    year_voor_re = re.compile(r'v[oó][oó]r ([12][0-9]{3})')
    year_match = re.findall(year_re, date)
    year_voor_match = re.findall(year_voor_re, date)
    if not year_match:
        return '0000'
    year = int(year_match[-1])
    if year_voor_match:
        year = year = 1
    return str(year)


## Add SQL records

This needs to be done in the right order. Records of which the recordid is needed in another table needs to be done first.

- builders needs nothing
- instruments needs nothing
- dispositions needs instruments, builders
- manuals needs dispositions
- stops needs manuals

Order:
- instruments
- builders
- dispositions
- manuals
- stops

The record ids will be added to the organdata for later reference.

## Create data structure to store the SQL records

In [13]:
#This will contain the SQL representations of the records
#NB Order matters (see above). This will be the order the records are inserted in the DB
def reset():
    records = {
        'instruments':               { 'nextid': 78692, 'records': [] },
        'builders':                  { 'nextid': 313, 'records': [] },
        'dispositions':              { 'nextid': 2947, 'records': [] },
        'manuals':                   { 'nextid': 5448, 'records': [] },
        'stops':                     { 'nextid': 40512, 'records': [] },
#        'countries':                 { 'nextid': 250, 'records': [] },
#        'couplers':                  { 'nextid': 0, 'records': [] },
#        'organizations':             { 'nextid': 10576, 'records': [] },
#        'people':                    { 'nextid': 0, 'records': [] },
#        'usage_types':               { 'nextid': 7, 'records': [] },
#        'additional_specifications': { 'nextid': 1616, 'records': [] },
#        'addresses':                 { 'nextid': 6808, 'records': [] },
#        'archive_instrument':        { 'nextid': 592, 'records': [] },
#        'archives':                  { 'nextid': 573, 'records': [] },
    }
    return records
records = reset()

## Instruments

In [14]:
#add entry to table 'instruments'
def add_instrument(organid):

    record_id = records['instruments']['nextid']
    records['instruments']['nextid'] += 1
    
    organnumber = toSQLstring(organdata['base'][organid]['organnumber'])
    if organnumber == "''":
        organnumber = 'NULL'
        
    monumentnumber = toSQLstring(organdata['base'][organid]['monumentnumber'])
    if monumentnumber == "''":
        monumentnumber = 'NULL'

    fields = OrderedDict([
        ('id',                   str(record_id)),
        ('name',                 toSQLstring(organdata['base'][organid]['name'])),
        ('description',          'NULL'),
        ('country',              toSQLstring('NL')),
        ('public',               str(0)),
        ('monument',             str(0)),
        ('organization_id',      'NULL'),
        ('owner_id',             'NULL'),
        ('physical_owner_id',    'NULL'),
        ('mentor_id',            'NULL'),
        ('created_at',           toSQLstring(time.strftime('%Y-%m-%d %H:%M:%S')) ),
        ('updated_at',           'NULL'),
        ('primary_image_id',     'NULL'),
        ('location_image_id',    'NULL'),
        ('deleted_at',           'NULL'),
        ('monument_number',      monumentnumber),
        ('organ_number',         organnumber),
        ('location_description', 'NULL'),
        ('case_description',     toSQLstring(texts_kunst[organid])),
        ('date_case_description','NULL'),
    ])

    records['instruments']['records'].append(
        "(" + ','.join(fields.values()) + ")"
    )

    return record_id

#do it
for organid in organids:
    record_id = add_instrument(organid)
    base[organid]['instrument_id'] = record_id



## Builders

Add builders to the table 'builders', and add record ids to the organdata

In [15]:
def add_builder(builder):
    builder_id = records['builders']['nextid']
    records['builders']['nextid'] += 1
    
    fields = OrderedDict([
        ('id',                 str(builder_id)),
        ('name',               toSQLstring(builder)),
        ('city',               'NULL'),
        ('country',            'NULL'),
        ('extra_information',  'NULL'),
        ('website',            'NULL'),
        ('active_from',        'NULL'),
        ('active_till',        'NULL'),
        ('created_at',         toSQLstring(time.strftime('%Y-%m-%d %H:%M:%S'))),
        ('updated_at',         'NULL'),
    ])
    records['builders']['records'].append(
        "(" + ','.join(fields.values()) + ")"
    )
    return builder_id

#builders are in: base info, and projects
for organid in organids:
    #building
    builder = history_base[organid]['builder']
    if builder != '':
        builder_id = add_builder(history_base[organid]['builder'])
        history_base[organid]['builder_id'] = builder_id
    else:
        history_base[organid]['builder_id'] = None

    #projects
    for proj in history_projects[organid]:
        builder = proj['name']
        if builder != '':
            builder_id = add_builder(builder)
            proj['builder_id'] = builder_id
        else:
            proj['builder_id'] = None


## Dispositions

add entries to table 'dispositions'. These are first delivery, projects, and dispositions!

### First Delivery

In [16]:
def add_first_delivery(organid):

    record_id = records['dispositions']['nextid']
    records['dispositions']['nextid'] += 1

    date = getSortingYear_disposition(organid, history_base[organid]['year'])
    date = date + '-01-01'

    builder_id = history_base[organid]['builder_id']
    if builder_id == None:
        builder_id = 'NULL'

    fields = OrderedDict([
        ('id',                        str(record_id)),
        ('date',                      toSQLstring(date)),
        ('estimate_date',             toSQLstring(history_base[organid]['year'])),
        ('history',                   toSQLstring("Year of delivery")),
        ('instrument_id',             str(base[organid]['instrument_id'])),
        ('builder_id',                str(builder_id)),
        ('created_at',                'NULL'),
        ('updated_at',                'NULL'),
        ('Particularities',           'NULL'),
        ('system_console',            'NULL'),
        ('system_playing_aids',       'NULL'),
        ('system_stop_action',        'NULL'),
        ('wind_system',               'NULL'),
        ('console_description',       'NULL'),
        ('playing_aids_description',  'NULL'),
        ('wind_system_description',   'NULL'),
        ('pitch',                     'NULL'),
        ('temperature',               'NULL'),
        ('console_location',          'NULL'),
        ('coupler_id',                'NULL'),
    ])

    records['dispositions']['records'].append(
        "(" + ','.join(fields.values()) + ")"
    )
    
    return record_id


### Projects

In [17]:
from cgi import print_arguments


def add_project(organid, project):

    record_id = records['dispositions']['nextid']
    records['dispositions']['nextid'] += 1

    date = getSortingYear_project(project['date'])
    date = date + '-01-01'

    history = '<p>-' + '<br />- '.join(project['changes']) + '</p>'

    builder_id = project['builder_id']
    if builder_id == None:
        builder_id = 'NULL'

    fields = OrderedDict([
        ('id',                        str(record_id)),
        ('date',                      toSQLstring(date)),
        ('estimate_date',             toSQLstring(project['date'])),
        ('history',                   toSQLstring(history)),
        ('instrument_id',             str(base[organid]['instrument_id'])),
        ('builder_id',                str(builder_id)),
        ('created_at',                toSQLstring(time.strftime('%Y-%m-%d %H:%M:%S')) ),
        ('updated_at',                'NULL'),
        ('Particularities',           'NULL'),
        ('system_console',            'NULL'),
        ('system_playing_aids',       'NULL'),
        ('system_stop_action',        'NULL'),
        ('wind_system',               'NULL'),
        ('console_description',       'NULL'),
        ('playing_aids_description',  'NULL'),
        ('wind_system_description',   'NULL'),
        ('pitch',                     'NULL'),
        ('temperature',               'NULL'),
        ('console_location',          'NULL'),
        ('coupler_id',                'NULL'),
    ])

    records['dispositions']['records'].append(
        "(" + ','.join(fields.values()) + ")"
    )

    return record_id



### Disposition

In [18]:
def add_disposition(organid, disposition):
    
    record_id = records['dispositions']['nextid']
    records['dispositions']['nextid'] += 1

    date = getSortingYear_disposition(
        organid,
        disposition['description']
    )
    date = date + '-01-01'

    estimated_date = disposition['description']

    fields = OrderedDict([
        ('id',                        str(record_id)),
        ('date',                      toSQLstring(date)), #e.g. 1773-01-01
        ('estimate_date',             toSQLstring(estimated_date)),
        ('history',                   toSQLstring(disposition['description'])),
        ('instrument_id',             str(base[organid]['instrument_id'])),
        ('builder_id',                'NULL'),
        ('created_at',                toSQLstring(time.strftime('%Y-%m-%d %H:%M:%S')) ),
        ('updated_at',                'NULL'),
        ('Particularities',           'NULL'),
        ('system_console',            'NULL'),
        ('system_playing_aids',       'NULL'),
        ('system_stop_action',        'NULL'),
        ('wind_system',               'NULL'),
        ('console_description',       'NULL'),
        ('playing_aids_description',  'NULL'),
        ('wind_system_description',   'NULL'),
        ('pitch',                     'NULL'),
        ('temperature',               'NULL'),
        ('console_location',          'NULL'),
        ('coupler_id',                'NULL'),
    ])

    records['dispositions']['records'].append(
        "(" + ','.join(fields.values()) + ")"
    )

    return record_id


### Current Disposition

In [19]:
def add_current_disposition(organid, disposition):
    
    record_id = records['dispositions']['nextid']
    records['dispositions']['nextid'] += 1

    date = str(part2date[organid[:6]])
    estimated_date = f'Huidige dispositie volgens Orgelencyclopedie {organid[:6]} ({date})'
    date = date + '-01-01'

    fields = OrderedDict([
        ('id',                        str(record_id)),
        ('date',                      toSQLstring(date)), #e.g. 2008-01-01
        ('estimate_date',             toSQLstring(estimated_date)),
        ('history',                   'NULL'),
        ('instrument_id',             str(base[organid]['instrument_id'])),
        ('builder_id',                'NULL'),
        ('created_at',                toSQLstring(time.strftime('%Y-%m-%d %H:%M:%S')) ),
        ('updated_at',                'NULL'),
        ('Particularities',           toSQLstring(texts_bijzonderheden[organid])),
        ('system_console',            'NULL'),
        ('system_playing_aids',       'NULL'),
        ('system_stop_action',        'NULL'),
        ('wind_system',               'NULL'),
        ('console_description',       'NULL'),
        ('playing_aids_description',  toSQLstring(tech[organid]['aids'])),
        ('wind_system_description',   toSQLstring(tech[organid]['windSys'])),
        ('pitch',                     toSQLstring(tech[organid]['pitch'])),
        ('temperature',               toSQLstring(tech[organid]['temperature'])),
        ('console_location',          toSQLstring(tech[organid]['consoleLoc'])),
        ('coupler_id',                'NULL'),
    ])

    records['dispositions']['records'].append(
        "(" + ','.join(fields.values()) + ")"
    )

    return record_id


### Add all dispositions

In [20]:
for organid in organids:

    add_first_delivery(organid)

    for proj in history_projects[organid]:
        record_id = add_project(organid, proj)
        proj['disposition_id'] = record_id
        
    for disp in dispositions[organid]['dispositions']:
        if disp['current']:
            record_id = add_current_disposition(organid, disp)
            disp['disposition_id'] = record_id
        else:
            record_id = add_disposition(organid, disp)
            disp['disposition_id'] = record_id   


## Manuals

In [21]:
#add entry to table 'instruments'
def add_manual(organid, disposition, division):

    record_id = records['manuals']['nextid']
    records['manuals']['nextid'] += 1
    
    #if current division: add keyboard ranges
    range = 'NULL'
    if disposition['current']:
        range = f"Klavieromvang: {tech[organid]['range1']};Pedaalomvang: {tech[organid]['range2']}"
        range = toSQLstring(range)

    fields = OrderedDict([
        ('id',               str(record_id)),
        ('name',             toSQLstring(division['division_name'])),
        ('description',      'NULL'),
        ('range',            range),
        ('wind_pressure',    'NULL'),
        ('windchest_system', 'NULL'),
        ('disposition_id',   str(disposition['disposition_id'])),
        ('created_at',       toSQLstring(time.strftime('%Y-%m-%d %H:%M:%S'))),
        ('updated_at',       'NULL'),
    ])

    records['manuals']['records'].append(
        "(" + ','.join(fields.values()) + ")"
    )

    return record_id

#do it
for organid in organids:
    for disp in dispositions[organid]['dispositions']:
        for division in disp['disposition']:
            record_id = add_manual(organid, disp, division)
            division['manual_id'] = record_id

## Stops

In [22]:
#add entry to table 'stops'
def add_stop(organid, disposition, division, stop):

    record_id = records['stops']['nextid']
    records['stops']['nextid'] += 1
    
    fields = OrderedDict([
        ('id',           str(record_id)),
        ('name',         toSQLstring(stop['stopname'])),
        ('partition',    toSQLstring(stop['partition'])),
        ('description',  toSQLstring(stop['footnote'])),
        ('value',        toSQLstring(stop['spec'])),
        ('voices',       'NULL'),
        ('manual_id',    str(division['manual_id'])),
    ])

    records['stops']['records'].append(
        "(" + ','.join(fields.values()) + ")"
    )

    return record_id

#do it
for organid in organids:
    for disp in dispositions[organid]['dispositions']:
        for division in disp['disposition']:
            for stop in division['stoplist']:
                record_id = add_stop(organid, disp, division, stop)
                stop['record_id'] = record_id

## Write to disk

In [23]:
#path to output data
outputroot = '../output/'

In [24]:
with open(os.path.join(outputroot,'data_encyclopedie.sql'), 'w') as f:
    for tablename, tabledata in records.items():
        f.write(f'LOCK TABLES `{tablename}` WRITE;\n')
        f.write(f'INSERT INTO `{tablename}` VALUES ')
        f.write(','.join(tabledata['records']).replace('\n','<br />'))
        f.write(';')
        f.write('\n')
        f.write('UNLOCK TABLES;\n\n')

